In [4]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


The goal of this is to learn tensorflow and hopefully learn how to get some good results on MNIST as well.

In [5]:
sess = tf.InteractiveSession()

## Softmax Regression Model

In [6]:
# placeholders (inputs to your model) 
x = tf.placeholder(dtype = tf.float32, shape = (None, 784),
                  name = 'input')
labels = tf.placeholder(dtype = tf.float32, shape = (None, 10),
                  name = 'output')
# variables (to be optimized during training)
# model is x dot W + b -- dimensions make sense
W = tf.Variable(initial_value = tf.zeros([784, 10]))
b = tf.Variable(initial_value = tf.zeros([10]))

sess.run(tf.global_variables_initializer()) # initializes global variables

In [7]:
#prediction based on model
unscaled_log_proba = tf.matmul(x, W) + b # softmax will be incuded in the 

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
        labels=labels,
        logits=unscaled_log_proba)
        )

# performs one optimization step
train_step = tf.train.GradientDescentOptimizer(0.5)\
            .minimize(loss)

In [8]:
n_iter = 1000
batch_size = 100

# 1000 times fetch a batch and update W and b
for _ in range(n_iter):
    batch = mnist.train.next_batch(batch_size)
    sess.run([train_step],feed_dict={x:batch[0], labels:batch[1]})

In [9]:
correct_prediction = tf.equal(tf.argmax(unscaled_log_proba, 1), tf.argmax(labels, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

[acc] = sess.run([accuracy], feed_dict={x:mnist.test.images, 
                                labels:mnist.test.labels})
print("We achieved an accuracy of {}%".format(acc))

We achieved an accuracy of 0.9172000288963318%


## CNN

In [10]:
def weight_variable(shape):
    # truncated normal: if more than 2SD above avg repick
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    #small positive number init to avoid dead ReLu neurons
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    return tf.nn.conv2d(input = x, filter = W, 
            strides = [1,1,1,1], padding = 'SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize = [1,2,2,1],
                strides = [1, 2, 2, 1], padding = 'SAME')

In [11]:
x_image = tf.reshape(x, shape = [-1, 28, 28, 1])

In [12]:
# first conv-layer
W_conv1 = weight_variable([5, 5, 1, 32]) # 5 by 5 by 32
b_conv1 = bias_variable([32]) # 1 num for each filter

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

# second conv-layer

# notice now volume is 32 layers deep
W_conv2 = weight_variable([5, 5, 32, 64]) 
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

# Densely Connected layer
# 1024 neurons in layer
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

flattened = tf.reshape(h_pool2, [-1, 7 * 7 * 64])
h_fc1 = tf.nn.relu(tf.matmul(flattened, W_fc1) + b_fc1)

# Add a dropout layer to prevent overfitting
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob = keep_prob)

# Output layer
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

In [13]:
# define loss and train
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    labels = labels, logits = y_conv
    ))

train_step = tf.train.AdadeltaOptimizer(1e-4).minimize(loss)

correct_prediction = tf.equal(tf.argmax(labels, 1), tf.argmax(y_conv, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [ ]:
sess.run(tf.global_variables_initializer())

batch_size = 50
n_iter = 1000

for _ in range(n_iter):
    batch = mnist.train.next_batch(batch_size)
    train_step.run(feed_dict={x:mnist.train.images, labels: mnist.train.labels, 
                              keep_prob: 0.5})
accuracy.eval(feed_dict={x:mnist.test.images, labels:mnist.test.labels, keep_prob:1.0})